In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install beautifulsoup4 #installing Beautiful Soup

from bs4 import BeautifulSoup 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 102kB 2.5MB/s ta 0:00:011
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Folium installed
Libraries imported.


## PART 1

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [6]:
canada_url = requests.get(url)

In [7]:
print(canada_url.status_code)

200


In [9]:
soup = BeautifulSoup(canada_url.text,"html.parser")

In [10]:
Table_array = []
Table_text_element = soup.find_all( class_= "wikitable sortable")
Table_text_element=Table_text_element[0]
for row in Table_text_element.find_all('tr'):
        for cell in row.find_all('td'):
             Table_array.append(cell.text)
Table_array1string=', '.join(Table_array)
Table_array1string = 'Column,PostCode,Borough,Neighbourhood' + '\n, '+ Table_array1string
fh = open ('Testfile.txt', 'w',)
fh.write(Table_array1string)
fh.close()
df = pd.read_csv('Testfile.txt')
#Remove the unwanted Column as part of Cleansing
df.drop('Column',axis=1,inplace=True)

In [11]:
df.head() #check dataset


,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_q1= df[df['Borough'] != ' Not assigned']
print(df_q1.shape)
df_q1.head()

(211, 3)


,PostCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [13]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
#Replacing Neighbourhood value with borough if its ' Not assigned'
df_q1.loc[df_q1['Neighbourhood'] == ' Not assigned', 'Neighbourhood'] = df_q1.Borough
print (df_q1.shape) #.shape method to print the number of rows of your dataframe

(211, 3)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
# more than one neighbourhood can exist in one postal code area + two rows will be combined into one row with the neighborhoods separated with a comma
res= df_q1.groupby(['PostCode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()
print("Number of rows in the final dataframe as per .Shape ", res.shape) #.shape method to print the number of rows of your dataframe

res.columns = ['Postal Code', 'Borough', 'Neighbourhood']
res.head() 
res.tail()

Number of rows in the final dataframe as per .Shape  (103, 3)


,Postal Code,Borough,Neighbourhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Ri..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergat..."
102,M9W,Etobicoke,Northwest


## PART 2 - WILL BE IN ANOTHER LINK